In [50]:
import os
import pandas as pd
import matplotlib.pyplot as plt

activities_file_name = os.path.join("activities.csv")
activities_file_name

'activities.csv'

In [62]:
activities = pd.read_csv(activities_file_name, index_col=0)
activities.rename(columns={'Activity Type': 'activity_type'}, inplace=True)
activities["Distance"] = pd.to_numeric(activities["Distance"], errors="coerce")
grouped_activities = activities.groupby("activity_type")
totals = grouped_activities.sum()
totals

,Elapsed Time,Distance,Relative Effort,Activity Gear
activity_type,,,,
Nordic Ski,250885,687.63,1440.0,0.0
Ride,339708,1716.82,6406.0,0.0
Rock Climbing,19800,0.00,0.0,0.0
Roller Ski,71213,228.38,252.0,0.0
Run,687429,2095.64,17587.0,0.0
Swim,16611,0.00,26.0,0.0
Weight Training,6420,0.00,0.0,0.0
Workout,54191,35.60,141.0,0.0
